# Credit risk modeling in Python - part 5

In [1]:
# Commands to get to same point for data in part 1 and 2 notebooks
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

data = pd.read_csv("creditrisk_pydata_nas.csv", index_col = 0)
data = data[data.age < 100]
data = data[data.children < 30]
data = data[data.address < (data.age + 2)]
data[(data.employer > (data.age - 14))]
data = data[(data.employer < (data.age - 14)) | data.employer.isnull()]
data.amount.fillna(data.amount.median(), inplace = True)

bins = [0, 10, 20 ,30, 40, 50]
bins_emp1 = pd.cut(data['employer'], bins)

bins = [0, 2, 5 ,10, 15, 50, 70]
bins_emp2 = pd.cut(data['employer'], bins)
bins_emp2.fillna("(50, 70]", inplace = True)
bins_emp2 = bins_emp2.cat.rename_categories(["(0, 2]","(2, 5]","(5, 10]","(10, 15]","(15, 50]", "unknown"])
bins_emp2 = bins_emp2.cat.as_unordered()

data['employer']= bins_emp2

data['payfreq'] = data['payfreq'].astype('category')
data['marstat'] = data['marstat'].astype('category')
data['home'] = data['home'].astype('category')

data['payfreq'] = data['payfreq'].cat.rename_categories(["quarter","bimon","monthly","biweek"])
data['marstat'] = data['marstat'].cat.rename_categories(["single","married","divorced","widowed"])
data['home'] = data['home'].cat.rename_categories(["yes", "no"])

data.dropna(inplace = True)

Create these for our newly "binned" categorical variable `employer`:
- A crosstable (value counts) --> store it in crosstab_employer
- A stacked chart
- A normalized crosstable

In [2]:
# Crosstable
crosstab_employer = pd.crosstab(data["employer"], data["default"])
crosstab_employer

default     0    1
employer          
(0, 2]    782  119
(2, 5]    734   71
(5, 10]   813   56
(10, 15]  387   16
(15, 50]  573   12
unknown   365   31

In [3]:
# Stacked chart
crosstab_employer.div(crosstab_employer.sum(1), axis=0).plot(kind='bar', stacked = True)
plt.title('Default for employment length')
plt.xlabel('Employment length')
plt.ylabel('Proportion of defaults')

In [4]:
# Crosstable with fractions
pd.crosstab(data["employer"], data["default"],  normalize='index')

default          0         1
employer                    
(0, 2]    0.867925  0.132075
(2, 5]    0.911801  0.088199
(5, 10]   0.935558  0.064442
(10, 15]  0.960298  0.039702
(15, 50]  0.979487  0.020513
unknown   0.921717  0.078283

## 3. One estimate for loan default

In [5]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics

In [6]:
data_ML = data[((data["default"] == 1) | (data["month"] == 24))]

In [7]:
print(data_ML.shape)

(2155, 10)


What changed?
- 2155 observations now. The default rate in this data set went up, from 7.7% to 14.15%
- Important assumption: distribution of the variables does not change drastically when taking out censored cases

### 3.1 Create dummies

In [8]:
from patsy import dmatrices

In [9]:
# create dataframes with an intercept column and dummy variables
y, X = dmatrices('default ~ age + amount + address + C(employer) + \
                  children + C(payfreq) + C(marstat) + C(home)',
                  data_ML, return_type = "dataframe")
print(X.columns)

Index(['Intercept', 'C(employer)[T.(2, 5]]', 'C(employer)[T.(5, 10]]',
       'C(employer)[T.(10, 15]]', 'C(employer)[T.(15, 50]]',
       'C(employer)[T.unknown]', 'C(payfreq)[T.bimon]',
       'C(payfreq)[T.monthly]', 'C(payfreq)[T.biweek]',
       'C(marstat)[T.married]', 'C(marstat)[T.divorced]',
       'C(marstat)[T.widowed]', 'C(home)[T.no]', 'age', 'amount', 'address',
       'children'],
      dtype='object')


In [10]:
y.head()

default
3      0.0
4      0.0
5      0.0
7      0.0
8      0.0

In [11]:
X.head()

Intercept  C(employer)[T.(2, 5]]  C(employer)[T.(5, 10]]  \
3        1.0                    0.0                     0.0   
4        1.0                    1.0                     0.0   
5        1.0                    0.0                     0.0   
7        1.0                    0.0                     1.0   
8        1.0                    1.0                     0.0   

   C(employer)[T.(10, 15]]  C(employer)[T.(15, 50]]  C(employer)[T.unknown]  \
3                      0.0                      0.0                     1.0   
4                      0.0                      0.0                     0.0   
5                      0.0                      0.0                     0.0   
7                      0.0                      0.0                     0.0   
8                      0.0                      0.0                     0.0   

   C(payfreq)[T.bimon]  C(payfreq)[T.monthly]  C(payfreq)[T.biweek]  \
3                  0.0                    1.0                   0.0   
4                  0.0                    1.0                   0.0   
5                  0.0                    1.0                   0.0   
7                  0.0                    1.0                   0.0   
8                  0.0                    1.0                   0.0   

   C(marstat)[T.married]  C(marstat)[T.divorced]  C(marstat)[T.widowed]  \
3                    0.0                     0.0                    0.0   
4                    0.0                     0.0                    0.0   
5                    0.0                     0.0                    0.0   
7                    1.0                     0.0                    0.0   
8                    0.0                     0.0                    0.0   

   C(home)[T.no]   age  amount  address  children  
3            1.0  21.0  1400.0      8.7       0.0  
4            1.0  21.0  1200.0     21.5       0.0  
5            1.0  62.0  1500.0      1.2       0.0  
7            0.0  52.0  5000.0     28.4       0.0  
8            1.0  32.0  1000.0      2.8       0.0

In [12]:
# fix column names of X
X = X.rename(columns = {'C(employer)[T.(2, 5]]':'emp_(2, 5]',
                        'C(employer)[T.(5, 10]]':'emp_(5, 10]',
                        'C(employer)[T.(10, 15]]':'emp_(10, 15]',
                        'C(employer)[T.(15, 50]]':'emp_(15, 50]',
                        'C(employer)[T.unknown]':'emp_unknown',
                        'C(payfreq)[T.bimon]':'freq_bimon',
                        'C(payfreq)[T.monthly]':'freq_monthly',
                        'C(payfreq)[T.biweek]':'freq_biweek',
                        'C(marstat)[T.married]': 'ms_married',
                        'C(marstat)[T.divorced]': 'ms_divorced',
                        'C(marstat)[T.widowed]':'ms_widowed', 
                        'C(home)[T.no]': 'home_no'})

In [13]:
X.head()

Intercept  emp_(2, 5]  emp_(5, 10]  emp_(10, 15]  emp_(15, 50]  \
3        1.0         0.0          0.0           0.0           0.0   
4        1.0         1.0          0.0           0.0           0.0   
5        1.0         0.0          0.0           0.0           0.0   
7        1.0         0.0          1.0           0.0           0.0   
8        1.0         1.0          0.0           0.0           0.0   

   emp_unknown  freq_bimon  freq_monthly  freq_biweek  ms_married  \
3          1.0         0.0           1.0          0.0         0.0   
4          0.0         0.0           1.0          0.0         0.0   
5          0.0         0.0           1.0          0.0         0.0   
7          0.0         0.0           1.0          0.0         1.0   
8          0.0         0.0           1.0          0.0         0.0   

   ms_divorced  ms_widowed  home_no   age  amount  address  children  
3          0.0         0.0      1.0  21.0  1400.0      8.7       0.0  
4          0.0         0.0      1.0  21.0  1200.0     21.5       0.0  
5          0.0         0.0      1.0  62.0  1500.0      1.2       0.0  
7          0.0         0.0      0.0  52.0  5000.0     28.4       0.0  
8          0.0         0.0      1.0  32.0  1000.0      2.8       0.0

In [14]:
# default (y) to array, for scikit-learn to properly understand it as a response
y = np.ravel(y)

### 3.2 Logistic regression

#### 3.2.1 Fitting a logistic regression model

- `statsmodels.api` when we look at parameters.
- scikit-learn afterwards.

In [15]:
import statsmodels.api as sm
logit_model = sm.Logit(y, X)
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.369322
         Iterations 7


In [16]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 2155
Model:                          Logit   Df Residuals:                     2138
Method:                           MLE   Df Model:                           16
Date:                Thu, 30 Nov 2017   Pseudo R-squ.:                 0.09421
Time:                        04:00:44   Log-Likelihood:                -795.89
converged:                       True   LL-Null:                       -878.67
                                        LLR p-value:                 6.472e-27
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept       -0.1000      0.317     -0.316      0.752        -0.720     0.520
emp_(2, 5]      -0.4435      0.171     -2.596      0.009        -0.778    -0.109
emp_(5, 10]     -0.6014      0.184     -3.263      0.001        -0.963    -0.240
emp_(10, 15]    -1.1479      0.290     -3.959      0.000        -1.716    -0.580
emp_(15, 50]    -1.6232      0.331     -4.903      0.000        -2.272    -0.974
emp_unknown     -0.4880      0.230     -2.119      0.034        -0.939    -0.037
freq_bimon      -0.1897      0.385     -0.492      0.623        -0.945     0.566
freq_monthly    -0.5853      0.136     -4.299      0.000        -0.852    -0.318
freq_biweek      0.7253      0.562      1.290      0.197        -0.377     1.828
ms_married      -0.0789      0.183     -0.430      0.667        -0.438     0.281
ms_divorced      0.5474      0.268      2.039      0.041         0.021     1.074
ms_widowed       0.5839      0.815      0.716      0.474        -1.014     2.182
home_no          0.3001      0.146      2.062      0.039         0.015     0.585
age             -0.0187      0.008     -2.212      0.027        -0.035    -0.002
amount       -4.717e-05   6.44e-05     -0.732      0.464        -0.000  7.91e-05
address         -0.0453      0.010     -4.407      0.000        -0.065    -0.025
children         0.0853      0.072      1.181      0.238        -0.056     0.227
================================================================================
"""

How does a 1 unit increase/decrease affect the odds of default?

In [17]:
np.exp(result.params)

Intercept       0.904877
emp_(2, 5]      0.641781
emp_(5, 10]     0.548017
emp_(10, 15]    0.317298
emp_(15, 50]    0.197275
emp_unknown     0.613843
freq_bimon      0.827194
freq_monthly    0.556911
freq_biweek     2.065345
ms_married      0.924139
ms_divorced     1.728750
ms_widowed      1.793050
home_no         1.349971
age             0.981426
amount          0.999953
address         0.955713
children        1.089003
dtype: float64

You'll notice that the results are slightly different in SK-learn. https://stats.stackexchange.com/questions/203740/logistic-regression-scikit-learn-vs-statsmodels

Parameters through scikit learn:

In [18]:
# logistic regression model on all the data using scikit learn (C = inverse regularization strength)
logreg = LogisticRegression(fit_intercept = False, C = 1e12)
model_log = logreg.fit(X, y)
model_log

LogisticRegression(C=1000000000000.0, class_weight=None, dual=False,
          fit_intercept=False, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [19]:
model_log.coef_

array([[ -1.10398342e-01,  -4.38962462e-01,  -6.05328755e-01,
         -1.14820612e+00,  -1.62173934e+00,  -4.89412887e-01,
         -2.02408005e-01,  -5.84734962e-01,   4.29731327e-01,
         -1.34366332e-01,   5.38176357e-01,   1.91702925e-01,
          3.00616585e-01,  -1.73694244e-02,  -5.06806869e-05,
         -4.55095478e-02,   9.55282520e-02]])

#### 3.2.2 Using the model to predict the probability of loan default

Predict the probability of default for someone who
Started a new job **last year**, repays **monthly**, is **widowed**, does **not own** a house, is **50 years old**, borrowed **4k USD**, has been living at the same **address for 24 years**, has **1 child**

In [20]:
print(X.columns)

Index(['Intercept', 'emp_(2, 5]', 'emp_(5, 10]', 'emp_(10, 15]',
       'emp_(15, 50]', 'emp_unknown', 'freq_bimon', 'freq_monthly',
       'freq_biweek', 'ms_married', 'ms_divorced', 'ms_widowed', 'home_no',
       'age', 'amount', 'address', 'children'],
      dtype='object')


In [21]:
predict_1= np.array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 50, 4000, 24, 1]).reshape(1, -1)

In [22]:
model_log.predict_proba(predict_1)

array([[ 0.90642191,  0.09357809]])

**Do it yourself!**

Predict the probability of default for someone who
- is at the same job for 8 years now
- Repays quarterly
- is married
- does not own a house
- is 38 years old
- borrowed 9k USD
- moved 2 years ago
- has 3 children

In [ ]:
predict_2 = ___
model_log.predict_proba(predict_2)